In [2]:
!pip install transformers==2.3.0

In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from sklearn.model_selection import train_test_split

# Load Huggingface transformers
from transformers import BertTokenizer, BertConfig
from transformers import TFBertModel

from tqdm.auto import tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import SGD

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [4]:
#reading the data
train_df= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')

In [5]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
test_labels[test_labels.toxic == 1]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
21,00091c35fa9d0465,1,0,0,0,0,0
48,0013fed3aeae76b7,1,0,1,0,1,1
59,0017d4d47894af05,1,0,1,0,1,0
76,001d739c97bc2ae4,1,0,0,0,0,0
81,001eff4007dbb65b,1,0,1,0,1,0
...,...,...,...,...,...,...,...
152879,ff84f0367ea58abb,1,0,1,0,1,0
152908,ff91c3d8a3e34398,1,0,0,0,1,0
153078,ffdf6854b41d9102,1,0,0,0,0,0
153119,ffebe90c8d5acaba,1,0,1,0,0,0


In [8]:
test_labels[test_labels.toxic == -1]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [9]:
target_col= train_df.columns[2:]
feature_col= train_df.columns[1:2]
target_col , feature_col

(Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
        'identity_hate'],
       dtype='object'),
 Index(['comment_text'], dtype='object'))

In [10]:
test_labels = test_labels.set_index('id')
test_labels

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,-1,-1,-1,-1,-1,-1
0000247867823ef7,-1,-1,-1,-1,-1,-1
00013b17ad220c46,-1,-1,-1,-1,-1,-1
00017563c3f7919a,-1,-1,-1,-1,-1,-1
00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...
fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1


In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-04-22 15:37:05.252776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 15:37:05.253840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 15:37:05.254516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [12]:
# Initial train and test split.
train_df, val_df = train_test_split(train_df,test_size=0.2)

In [13]:
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 127656
Number of rows in validation set: 31915
Number of rows in test set: 153164


In [14]:
def data_tokenization(dataset,max_len,tokenizer):
    tokens = tokenizer.batch_encode_plus(dataset['comment_text'].to_list(), max_length=max_len, add_special_tokens=True,return_tensors='tf', pad_to_max_length=True)
    input_ids, attention_mask=np.squeeze(tokens['input_ids']), np.squeeze(tokens['attention_mask'])

    return [input_ids,attention_mask]

In [15]:
X_train = train_df[feature_col]
y_train = train_df[target_col]

X_val = val_df[feature_col]
y_val = val_df[target_col]

In [16]:
X_train.head()

,comment_text
96245,Example you asked for\n\nYou very kindly asked...
99930,No need to kick them where they are down. Your...
119486,Why is there a random picture of notation in t...
42587,I got a message about vandilism. I did not do ...
2256,"""\n\n One Addition \n\nMustafaa, I promised mu..."


In [17]:
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
96245,0,0,0,0,0,0
99930,0,0,0,0,0,0
119486,0,0,0,0,0,0
42587,0,0,0,0,0,0
2256,0,0,0,0,0,0


In [18]:
num_labels = y_train.shape[1]
print(num_labels)

6


In [19]:
#######################################
### --------- Setup BERT ---------- ###

# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 128

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name, from_pt=False)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = model_name, config = config, from_pt=False, do_lower_case=True)

# Load BERT model
base_model = TFBertModel.from_pretrained(pretrained_model_name_or_path = model_name, config = config, from_pt=False)

2022-04-22 15:37:30.556489: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 15:37:30.558413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 15:37:30.559239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-22 15:37:30.559866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [20]:
X_train = data_tokenization(X_train,max_length,tokenizer)

In [21]:
X_val = data_tokenization(X_val,max_length,tokenizer)

In [22]:
X_test = data_tokenization(test_df,max_length,tokenizer)

In [23]:
print(f"Shape of training set: {X_train[0].shape}")
print(f"Shape of validation set: {X_val[0].shape}")
print(f"Shape of test set: {X_test[0].shape}")

Shape of training set: (127656, 128)
Shape of validation set: (31915, 128)
Shape of test set: (153164, 128)


In [24]:
def ToxicClassificationModel(bert_base_model, num_labels, max_length):

    input_ids= Input(shape=(max_length,),dtype=tf.int32,name='input_ids')
    attention_mask=Input(shape=(max_length,),dtype=tf.int32,name='attention_mask')

    bert= bert_base_model.bert(input_ids, attention_mask=attention_mask)
    x=bert[0][:,0,:]
    x=tf.keras.layers.Dropout(0.1)(x)
    x=tf.keras.layers.Dense(128)(x)
    x=tf.keras.layers.Dense(64)(x)
    x=tf.keras.layers.Dense(32)(x)

    output=tf.keras.layers.Dense(num_labels,activation='sigmoid')(x)

    model=Model(inputs=[input_ids,attention_mask],outputs=[output])
    
    return model

In [25]:
model = ToxicClassificationModel(base_model, num_labels, max_length)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          ((None, 128, 768), ( 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                   

In [27]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy')
acc = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

In [28]:
model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [29]:
tf.keras.backend.clear_session()

In [30]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=1
)

2022-04-22 15:48:37.999923: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


7979/7979 [==============================] - 2290s 285ms/step - loss: 0.0460 - accuracy: 0.7741 - auc: 0.9839 - recall: 0.6934 - precision: 0.8059 - val_loss: 0.0391 - val_accuracy: 0.9398 - val_auc: 0.9883 - val_recall: 0.7714 - val_precision: 0.7934


In [81]:
X_test[0].shape

(153164, 128)

In [82]:
test_labels.shape

(153164, 6)

In [95]:
val_df[feature_col].shape

(31915, 1)

In [128]:
y_val_pred = model.predict(X_val)

In [129]:
y_sub_pred = model.predict(X_test)

In [130]:
y_val_pred.shape

(31915, 6)

In [131]:
y_sub_pred.shape

(153164, 6)

In [132]:
from sklearn.metrics import classification_report

In [133]:
print(classification_report(y_val, y_val_pred.round(), target_names=target_col))

               precision    recall  f1-score   support

        toxic       0.85      0.80      0.83      3072
 severe_toxic       0.49      0.38      0.43       307
      obscene       0.83      0.84      0.84      1662
       threat       0.44      0.72      0.55       111
       insult       0.74      0.79      0.76      1572
identity_hate       0.73      0.37      0.50       281

    micro avg       0.79      0.77      0.78      7005
    macro avg       0.68      0.65      0.65      7005
 weighted avg       0.80      0.77      0.78      7005
  samples avg       0.07      0.07      0.07      7005



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [136]:
sample_df= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [137]:
sample_df

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.5,0.5,0.5,0.5,0.5,0.5
153160,fffd7a9a6eb32c16,0.5,0.5,0.5,0.5,0.5,0.5
153161,fffda9e8d6fafa9e,0.5,0.5,0.5,0.5,0.5,0.5
153162,fffe8f1340a79fc2,0.5,0.5,0.5,0.5,0.5,0.5


In [157]:
submiss_df= pd.DataFrame(y_sub_pred, columns= target_col)
submiss_df['id']=test_labels.index

In [159]:
submiss_df = submiss_df[['id']+target_col.tolist()]
submiss_df

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996038,0.416804,0.986507,0.501910,0.969807,0.143741
1,0000247867823ef7,0.000138,0.000006,0.000011,0.000011,0.000015,0.000007
2,00013b17ad220c46,0.000470,0.000042,0.000046,0.000092,0.000069,0.000053
3,00017563c3f7919a,0.000028,0.000016,0.000009,0.000031,0.000006,0.000007
4,00017695ad8997eb,0.000383,0.000013,0.000019,0.000026,0.000025,0.000006
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.831173,0.008366,0.574746,0.000677,0.149024,0.003019
153160,fffd7a9a6eb32c16,0.004529,0.000087,0.000295,0.000096,0.000459,0.000102
153161,fffda9e8d6fafa9e,0.000055,0.000013,0.000036,0.000009,0.000009,0.000021
153162,fffe8f1340a79fc2,0.000577,0.000072,0.000069,0.000121,0.000124,0.000882


In [164]:
submiss_df.to_csv('submission_final.csv', index=False)